# Scraping MERISTATION

In [16]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import newspaper
from newspaper import Article
from newspaper import Source
from newspaper import fulltext

## Scraping Articles Site 

In [18]:
review_url = 'https://as.com/meristation/analisis/'
html = requests.get(review_url).content
html[:50]

b'<!DOCTYPE html>\n<html lang="es">\n<head>\n<meta char'

In [19]:
soup = BeautifulSoup(html, 'lxml')
articles = soup.find_all('h2')

In [20]:
#Extraer títulos

titles = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        titles.append(a.find('a')['title'])
    
titles[0:5]

['art of rally, análisis',
 'Crash Bandicoot 4, Análisis. Celebración plataformera',
 'WWE 2K Battlegrounds, análisis. La lucha libre nos vuelve a romper el corazón',
 'Turtle Beach Stealth 700 Gen 2, Análisis',
 'Maid of Sker, análisis: el fan victoriano de Resident Evil VII']

In [21]:
#Extraer links

links = []

for a in articles:
    if a.find('a') == None:
        pass
    else:
        links.append(a.find('a')['href'])
    
links[0:6]

['https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html',
 'https://as.com/meristation/2020/10/01/analisis/1601557244_384472.html',
 'https://as.com/meristation/2020/09/30/analisis/1601454855_979194.html',
 'https://as.com/meristation/2020/09/29/mexico/1601416248_335912.html',
 'https://as.com/meristation/2020/09/29/analisis/1601370434_254241.html',
 'https://as.com/meristation/2020/09/29/analisis/1601367548_437006.html']

## Page parsing and link retrieving function for Meristation

In [22]:
def meristation_link_retrieve(num_pages):
    links = []
    
    # pages parser
    for i in range(num_pages):
        url = f'https://as.com/meristation/analisis/{331-i}'
        #print(url)

        # building soup    
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        articles = soup.find_all('h2')

        # links retrieve
        for a in articles:
            if a.find('a') == None:
                pass
            else:
                links.append(a.find('a')['href'])
        
    #print(links)
    
    #Deleting non review links
    for link in links:
        if 'analisis' not in link:
            links.remove(link)
                
    return links


In [23]:
num_pages = 2

links = meristation_link_retrieve(num_pages)

print(links)

['https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html', 'https://as.com/meristation/2020/10/01/analisis/1601557244_384472.html', 'https://as.com/meristation/2020/09/30/analisis/1601454855_979194.html', 'https://as.com/meristation/2020/09/29/analisis/1601370434_254241.html', 'https://as.com/meristation/2020/09/29/analisis/1601367548_437006.html', 'https://as.com/meristation/2020/09/28/analisis/1601283612_640735.html', 'https://as.com/meristation/2020/09/26/analisis/1601118163_185827.html', 'https://as.com/meristation/2020/09/26/analisis/1601109660_140327.html', 'https://as.com/meristation/2020/09/25/analisis/1601038731_079093.html', 'https://as.com/meristation/2020/09/25/analisis/1601019685_445862.html', 'https://as.com/meristation/2020/09/24/analisis/1600959846_926566.html', 'https://as.com/meristation/2020/09/23/analisis/1600848195_838475.html', 'https://as.com/meristation/2020/09/22/analisis/1600764887_024298.html', 'https://as.com/meristation/2020/09/22/analisis/16

## Scraping Single Review

In [212]:
review_url = 'https://as.com/meristation/2020/09/29/analisis/1601370434_254241.html'
review_html = requests.get(review_url).content
review_html[:50]

b'<!DOCTYPE html>\n<html lang="es">\n<head>\n<meta char'

In [213]:
soup = BeautifulSoup(review_html, 'lxml')

In [214]:
p_tags = soup.find('div', {'class': 'art-body'}).find_all('p')

In [215]:
review = [tag.text for tag in p_tags]
review = ' '.join(review)

In [216]:
review[0:1000]

'"Un consejo, señor, no se acerque nunca al lago... Y sobre todo, tápese los oídos si oye cantar la voz bajo el agua...", El fantasma de la ópera" (1910), Gastón Leroux En un videojuego, lo mismo que en una película, poco importa si es grande o pequeño, si lo han hecho un estudio de 100 personas con 80 millones de presupuesto, o 5 con apenas unas decenas de miles. Lo que cuenta es su eficacia, su capacidad para lograr el fin último que su género busca. Maid of Sker logra eso en su estupenda primera parte, pero lo arruina en su segunda, en un juego con 2 mitades diferenciadas por sus mecánicas. Pero incluso tal como está, su historia merece ser descubierta. Tras un Don’t Knock Twice que ciertamente componía una gran atmósfera, el estudio Wales Interactive regresa este año con Maia of Sker, un título con el que buscan solventar los fallos del anterior al tiempo que contarnos una estupenda historia de horror cósmico - marítimo. El punto de partida es directo: Estás en el siglo XIX, en un 

In [218]:
author = soup.find('li', {'class': 'art-aut-wr'}).text.strip('\n')
author

'César Otero'

In [219]:
# Author
def meri_author():
    author = 'None'
    
    try:
        author = soup.find('p', {'class': 'art-aut-wr'}).find('a').text

    except AttributeError:
        pass
    
    try:
        author = soup.find('li', {'class': 'art-aut-wr'}).text.strip('\n')

    except AttributeError:
        pass

    return author

In [191]:
# Game

'''News_article = Article(review_url)
News_article.download()
News_article.parse()
News_article.title'''

game = soup.find('div', {'class': 'ga-h-tl'}).text
game

'WWE 2K Battlegrounds'

In [192]:
# Genre, Company and Platform

infobox = soup.find('ul', {'class': 'li-inl'}).contents
for box in infobox:  
        
    if (type(box.find('span')) == int) or (box.find('span') == None):
        pass
    else:
        #print(box.find('span').text)
 
        if 'Plataforma' in box.find('span').text:
            group = box.find_all('a', {'class': 'rv-inline'})
            platform = [plat.contents[0] for plat in group]
            platform = ' '.join(platform)
            
        if 'Género' in box.find('span').text:
            genre = box.find('span', {'class': 'val'}).text
            
        if 'Editor' in box.find('span').text:
            company = box.find('span', {'class': 'val'}).text

print(platform, genre, company)


NSW
 
STD
 
PS4
 
XBO
 
PC
 
Acción                     
2K Games



In [204]:
# Score

def meri_score():
    #Initialize with most common score
    score = 7

    #3 different paths for score in Meristation
    try:
        score = soup.find('span', {'class': 'rv-sc sc-h'}).text

    except AttributeError:
        pass

    try:
        score = soup.find('span', {'class': 'rv-sc sc-m'}).text

    except AttributeError:
        pass

    try:
        score = soup.find('span', {'class': 'rv-sc sc-l'}).text

    except AttributeError:
        pass
   
    return score

5.7


## Create columns

In [223]:

def meristation_dict(links):
    reviews_dict = {}
    i = 0

    for link in links:
        print(link)

        #Request content and wait
        review_html = requests.get(link).content
        
        #Avoid get banned and timeout
        time.sleep(1)

        #Make a Soup and a Newspacer build
        soup = BeautifulSoup(review_html, 'lxml')
        paper = newspaper.build(link)

        #Site name - From Newspaper
        site = paper.brand

        #URL - From Newspaper
        url_link = paper.url
        
        #Game - From Scraping
        game = soup.find('div', {'class': 'ga-h-tl'}).text
        print(game)

        #Author - From Scraping
        author = meri_author() 
        
        #Company, Genre & Platform - From Scraping
        genre = 'None'
        company = 'None'
        platform = 'None'
        
        infobox = soup.find('ul', {'class': 'li-inl'}).contents
        for box in infobox:  

            if (type(box.find('span')) == int) or (box.find('span') == None):
                pass
            else:
                #print(box.find('span').text)

                if 'Plataforma' in box.find('span').text:
                    group = box.find_all('a', {'class': 'rv-inline'})
                    platform = [plat.contents[0] for plat in group]
                    platform = ' '.join(platform).strip('\n')

                if 'Género' in box.find('span').text:
                    genre = box.find('span', {'class': 'val'}).text.strip('\n')

                if 'Editor' in box.find('span').text:
                    company = box.find('span', {'class': 'val'}).text.strip('\n')

        #Text & Cleaning - From Scraping
        p_tags = soup.find('div', {'class': 'art-body'}).find_all('p')
        review = [tag.text for tag in p_tags]
        review = ' '.join(review)

        #Score & Clean & Transform - From Scraping
        score = meri_score()
        score = float(score)

        #Add to a dict
        reviews_dict[i] = {'site': site,
                           'url_link': url_link,
                           'author': author,
                           'game': game,
                           'company': company,
                           'genre': genre,
                           'platform': platform,
                           'text': review,
                           'score': score}
        
        i = i+1

    return reviews_dict

In [224]:
result = meristation_dict(links[0:5])
print(result)

https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html
art of rally
https://as.com/meristation/2020/10/01/analisis/1601557244_384472.html
Crash Bandicoot 4: It's About Time
https://as.com/meristation/2020/09/30/analisis/1601454855_979194.html
WWE 2K Battlegrounds
https://as.com/meristation/2020/09/29/analisis/1601370434_254241.html
Maid of Sker
https://as.com/meristation/2020/09/29/analisis/1601367548_437006.html
Pathfinder: Kingmaker
{0: {'site': 'as', 'url_link': 'https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html', 'author': 'César Otero', 'game': 'art of rally', 'company': 'Funselektor', 'genre': 'Conducción                    ', 'platform': 'PC', 'text': 'Los fans de los videojuegos de carreras están disfrutando de un verano de lo más movidito; y es que además de las recientes entregas anuales de sagas como WRC o F1, la tercera parte de Project Cars y los arcade Trackmania Remake, Inertial Drift o Hotshot Racing, ahora debemos sumar a la ecuación 

In [225]:
result[0]

{'site': 'as',
 'url_link': 'https://as.com/meristation/2020/10/03/analisis/1601701228_240717.html',
 'author': 'César Otero',
 'game': 'art of rally',
 'company': 'Funselektor',
 'genre': 'Conducción                    ',
 'platform': 'PC',
 'text': 'Los fans de los videojuegos de carreras están disfrutando de un verano de lo más movidito; y es que además de las recientes entregas anuales de sagas como WRC o F1, la tercera parte de Project Cars y los arcade Trackmania Remake, Inertial Drift o Hotshot Racing, ahora debemos sumar a la ecuación un título muy esperado por los fans de los rallys con art of rally, una curiosa y original propuesta a medio camino entre la simulación más exigente y los arcade de siempre que pretende hacernos revivir la época dorada de tan popular categoría de automovilismo, desde sus inicios en la década de los 60 hasta la explosión del famoso Grupo B de los 80, un recorrido histórico a cago de los canadienses Funselektor, creadores del celebrado y no menos or

### SCRAPING REVIEWS

### Create DataFrame

In [33]:
meristation = pd.DataFrame({'title': titles, 
                           'author': np.nan, 
                           'review': reviews_col, 
                           'url': links 
                           })

meristation.head()

,title,author,review,url
0,Análisis Bounty Battle; el título de lucha de ...,NaN,Después del anuncio de su retraso indefinido e...,https://as.com/meristation/2020/09/22/analisis...
1,"eFootball PES 2021, análisis. Rueda el balón, ...",NaN,La temporada futbolística acabó de manera atíp...,https://as.com/meristation/2020/09/22/analisis...
2,"Milky Way Prince: The Vampire Prince, análisis...",NaN,El amor es uno de los temas universales que pe...,https://as.com/meristation/2020/09/21/analisis...
3,"Spinch, análisis PC y Switch: Una experiencia ...",NaN,No hay duda de que el género de las plataforma...,https://as.com/meristation/2020/09/20/analisis...
4,"Spelunky 2, análisis: el videojuego más hardco...",NaN,Mucho ha llovido desde el lanzamiento de Spelu...,https://as.com/meristation/2020/09/19/analisis...


In [34]:
#meristation.to_csv('../data/meristation_data.csv', index=False)